In [1]:
######### This is the code for normal (single objective) meet in the middle algorithm applied to the uncertain qz problem ################

import numpy as np
import matplotlib.pyplot as plt
from utils_MM import GraphConstructionDiscretization
import time
import networkx as nx
import heapq


In [2]:
# =============================================================================
# Constants, UAV parameters, and initial conditions
# =============================================================================
# Define QZ circles as tuples (x, y, radius, only_electric_radius, risk_limit, toggle_only_electric_or_no_path) toggle_only_electric_or_no_path = 0 for no path, 1 for only electric

# Map with two QZ circles
# map_qz = [(0.0, 0.0, 6.0, 2.0, 30, 1), (12.0, 10.0, 4.0, 1.5, 30, 1), (-5.0, -8.0, 3.0, 0.5, 15, 1), (17.5, 22.5, 7.0, 2.5, 40, 1), (0, 18.0, 6.0, 1.5, 25, 1) ]
# start = (-10, -10)       # Starting point
# goal = (25, 25)        # Goal point

map_qz = [(0.0, 0.0, 6.0, 2.0, 30, 1), (12.0, 10.0, 4.0, 1.5, 30, 1) ]
start = (-5, -5)       # Starting point
goal = (15, 15)        # Goal point
# max_risk_limit = 2 / 3 * sum([circle[-2] for circle in map_qz]) ### Total risk limit i.e. limit on the sum of all the risk limits
# acceptable_risk_limit = 1 / 3 * sum([circle[-2] for circle in map_qz])


# UAV characteristics
alpha = 10                      ### Discharge rate 
recharge_factor = 2             ### Factor by which recharge rate less than discharge rate
beta = alpha / recharge_factor  ### Recharge rate

q_min, q_max, q_act = 20, 100, 70 ### Minimum SOC limit, maximum SOC limit, and SOC at the state
discretization_angle = 10         ### Discretization angle for QZ in degrees

In [3]:
# =============================================================================
# Graph Construction and Node Creation
# =============================================================================
# Instantiate the graph construction object.
graph_object = GraphConstructionDiscretization(map_qz, start, goal, q_min, q_max, q_act, alpha, beta, discretization_angle)

start_time = time.time()

# Create nodes ( x_pos, y_pos, qz_index, SOC) for the graph 
# Create the index maps and reverse index maps mapping nodes to node index. 
nodes, index_map, reverse_index_map = graph_object.create_nodes()

# Build the visibility graph
graph_object.build_visibility_graph(reverse_index_map)
end_time = time.time()

print(f"The time req for graph construction: {end_time-start_time}")

## Assign heuristic cost to each node
graph_object.assign_heuristic_costs(reverse_index_map)

The time req for graph construction: 0.5469050407409668


In [7]:
def Meet_in_the_middle(graph_object):
    #### Meet in the middle algorithm for single objective
    visibility_graph = graph_object.visibility_graph
    all_states = list(visibility_graph.nodes)
    U = np.inf
    start_state = "s"
    goal_state = "g"

    open_F, open_B, closed_F, closed_B = [], [], [], []
    start_node = (0, 0, 0, start_state) ### max(f, 2g), f, g, state 
    goal_node =  (0, 0, 0, goal_state)   ### max(f, 2g), f, g, state

    heapq.heappush(open_F, [start_node, (None, None, None, None)])
    heapq.heappush(open_B, [goal_node, (None, None, None, None)])

    Prmin_F, Prmin_B = np.inf, np.inf

    g_F_list = {state: np.inf for state in all_states}
    g_B_list = {state: np.inf for state in all_states}

    while open_F or open_B:
        # print("Inside the while loop")
        if open_F:
            Prmin_F, _, _ , _ = open_F[0][0]
        else:
            Prmin_F = None
        if open_B:
            Prmin_B, _, _ , _ = open_B[0][0]
        else:
            Prmin_B = None
        
        C = min(Prmin_F, Prmin_B)
        # print(f"Prmin_F: {Prmin_F}, Prmin_B: {Prmin_B}, C: {C}")
        F_min_F = min([node[0][1] for node in open_F])
        F_min_B = min([node[0][1] for node in open_B])
        G_min_F = min([node[0][2] for node in open_F])
        G_min_B = min([node[0][2] for node in open_B])
        
        if U <= max(C, F_min_F, F_min_B, G_min_F + G_min_B):
            return U, open_F[0], open_B[0]
        
        if C == Prmin_F:
            # print("Inside forward search")
            current_node, parent_node = heapq.heappop(open_F)
            current_pr, current_f, current_g, current_state = current_node
            closed_F.append(current_state)
            
            for succ in visibility_graph.successors(current_state):
                # print("Inside forward search for successors")
                open_F_list = [node[0][-1] for node in open_F]
                edge_data = visibility_graph.edges[current_state, succ]
                
                if (succ in closed_F or succ in open_F_list) and (g_F_list[succ] <= g_F_list[current_state] + edge_data["fuel_cost"]):
                    continue
                
                if (succ in closed_F or succ in open_F_list):
                    closed_F = [node for node in closed_F if node != succ]
                    open_F = [node for node in open_F if node[0][-1] != succ]
                    heapq.heapify(open_F)
                
                g_succ = current_g + edge_data["fuel_cost"]
                f_succ = g_succ + visibility_graph.nodes[succ]['heuristic_cost']["fuel_cost"]
                g_F_list[succ] = g_succ
                pr_succ = max(f_succ, 2*g_F_list[succ])
                # print(f"pr_succ: {pr_succ}, f_succ: {f_succ}, g_succ: {g_succ}, succ: {succ}")
                heapq.heappush(open_F, [(pr_succ, f_succ, g_succ, succ), current_node])
                
                open_B_list = [node[0][-1] for node in open_B]
                
                if succ in open_B_list:
                    U = min(U, g_F_list[succ] + g_B_list[succ])
        else:
            # print("Inside backward search")
            current_node, parent_node = heapq.heappop(open_B)
            current_pr, current_f, current_g, current_state = current_node
            closed_B.append(current_state)
            
            for succ in visibility_graph.successors(current_state):
                open_B_list = [node[0][-1] for node in open_B]
                edge_data = visibility_graph.edges[current_state, succ]
                
                if (succ in closed_B or succ in open_B_list) and (g_B_list[succ] <= g_B_list[current_state] + edge_data["fuel_cost"]):
                    continue
                
                if (succ in closed_B or succ in open_B_list):
                    closed_B = [node for node in closed_B if node != succ]
                    open_B = [node for node in open_B if node[0][-1] != succ]
                    heapq.heapify(open_B)
                
                g_succ = current_g + edge_data["fuel_cost"]
                f_succ = g_succ + visibility_graph.nodes[succ]['heuristic_cost']["fuel_cost"]
                g_B_list[succ] = g_succ
                pr_succ = max(f_succ, 2 * g_B_list[succ])
                
                heapq.heappush(open_B, [(pr_succ, f_succ, g_succ, succ), current_node])
                
                open_F_list = [node[0][-1] for node in open_B]
                if succ in open_F_list:
                    U = min(U, g_F_list[succ] + g_B_list[succ])


In [8]:
start = time.time()
sol = Meet_in_the_middle(graph_object)
end = time.time()
print(f"The time req for Meet in the middle: {end-start}")

The time req for Meet in the middle: 0.03042459487915039
